In [1]:
from functools import partial
import scipy.sparse as sparse
import multiprocessing as mp
import pandas as pd
import numpy as np
import tqdm
import copy
import gc
import os
import sys
sys.path.append('../../../code/utils')
sys.path.append('../../../code/pipeline')
sys.path.append('../../../code')
import data_utils as du
import perf_utils as pu
import data_jointer as dj
import config

In [2]:
pairs = [# ('productId', 'LBS'),  # no need in this case
         ('productType', 'kw1'),  # 'kw1' looks very overfitting prone, to be decide whether to keep it
         ('productType', 'kw2'),
         ('productType', 'kw3'),
         ('productType', 'topic1'),
         ('aid', 'topic2'),
         ('productType', 'topic2'),
         # ('productType', 'topic3'),  # might help in predicting negative samples
         # ('productType', 'appIdInstall'),  # might help in predicting negative samples
         # ('productType', 'appIdAction'),  # might help in predicting negative samples
         ('advertiserId', 'interest1'),
         ('aid', 'interest2'),
         ('creativeSize', 'interest2'), 
         ('campaignId', 'interest4'),  # whether to keep it? 
         ('aid', 'interest5'),  
         ('aid', 'ct'),
         ('aid', 'os')]

In [3]:
def clickrate_stack_path(ad_feat_name, user_feat_name, dataset="train"):
    stack_folder = os.path.join(config.DATA_DIR, "stacking/clickrate")
    stack_file = "{}.[adFeatureName='{}'][userFeatureName='{}'].pkl".format(dataset, ad_feat_name, user_feat_name)
    stack_path = os.path.join(stack_folder, stack_file)
    return stack_path

In [7]:
def resave(ad_feat_name, user_feat_name, dataset="train"):
    stack_path = clickrate_stack_path(ad_feat_name, user_feat_name, dataset)
    df = du.load_pickle(stack_path)
    cols = df.columns.tolist()
    matrix = df.values
    
    assert matrix.shape[0] == df.shape[0]
    assert matrix.shape[1] == df.shape[1]
    assert len(cols) == df.shape[1]
    du.save_pickle((cols, matrix), stack_path)

In [10]:
for ad_feat_name, user_feat_name in pairs:
    with pu.profiler("resaving '{}' x '{}' features"):
        resave(ad_feat_name, user_feat_name, "train")
        resave(ad_feat_name, user_feat_name, "test1")

['bsClickrate@productType_x_kw1_q0',
 'bsClickrate@productType_x_kw1_q25',
 'bsClickrate@productType_x_kw1_q50',
 'bsClickrate@productType_x_kw1_q75',
 'bsClickrate@productType_x_kw1_q100',
 'bsClickrate_weighted_avg@productType_x_kw1',
 'bsClickrate@productType_x_kw1_mean',
 'bsClickrate@productType_x_kw1_std',
 'impression@productType_x_kw1_mean',
 'impression@productType_x_kw1_std',
 'max_bsClickrate_impression@productType_x_kw1',
 'min_bsClickrate_impression@productType_x_kw1']